In [ ]:
!pip install opencv-python mediapipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 16.5 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires flatbuffers<2,>=1.12, but you have flatbuffers 23.1.4 which is incompatible.


In [ ]:
import cv2
import mediapipe as mp

In [ ]:
from PIL import Image
import numpy as np
import pandas as pd

In [ ]:
from scipy import stats

In [ ]:
import copy

In [ ]:
import math

In [ ]:
import colorsys

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

In [ ]:
from google.colab.patches import cv2_imshow

image = cv2.imread("1.jpeg")
image

In [ ]:
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
with mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5) as face_mesh:
    image = cv2.imread("1.jpeg")
    results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    annotated_image = image.copy()
    for face_landmarks in results.multi_face_landmarks:
      print('face_landmarks:', face_landmarks)
      mp_drawing.draw_landmarks(
          image=annotated_image,
          landmark_list=face_landmarks,
          connections=mp_face_mesh.FACEMESH_TESSELATION,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp_drawing_styles
          .get_default_face_mesh_tesselation_style())
      mp_drawing.draw_landmarks(
          image=annotated_image,
          landmark_list=face_landmarks,
          connections=mp_face_mesh.FACEMESH_CONTOURS,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp_drawing_styles
          .get_default_face_mesh_contours_style())
      mp_drawing.draw_landmarks(
          image=annotated_image,
          landmark_list=face_landmarks,
          connections=mp_face_mesh.FACEMESH_IRISES,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp_drawing_styles
          .get_default_face_mesh_iris_connections_style())
      cv2.imwrite('/tmp/annotated_image' + '.png', annotated_image)

In [ ]:
for face in results.multi_face_landmarks:
    for landmark in face.landmark:
        x = landmark.x
        y = landmark.y

        shape = image.shape
        relative_x = int(x * shape[1])
        relative_y = int(y * shape[0])

        cv2.circle(image, (relative_x, relative_y), radius=1, color=(225, 0, 100), thickness=1)
cv2_imshow(image)

In [ ]:
def coordinates (landmark1, landmark2, image):
  data=np.array(image)
  shape=data.shape
  x1=int((results.multi_face_landmarks[0].landmark[landmark1].x)*shape[1])
  y1=int((results.multi_face_landmarks[0].landmark[landmark1].y)*shape[0])
  x2=int((results.multi_face_landmarks[0].landmark[landmark2].x)*shape[1])
  y2=int((results.multi_face_landmarks[0].landmark[landmark2].y)*shape[0])
  xmin=min(x1,x2)
  xmax=max(x1,x2)
  ymin=min(y1,y2)
  ymax=max(y1,y2)
  return (xmin,ymin,xmax, ymax)


In [ ]:
image=Image.open("1.jpg")

In [ ]:
data=np.array(image)
shape=data.shape

In [ ]:
forehead_coordinates=coordinates(333, 104, image)
nose_coordinates=coordinates (2,6,image)
#cheek_coordinates=coordinates (11,20, image)

In [ ]:
forehead_crop=image.crop(forehead_coordinates)
nose_crop=image.crop(nose_coordinates)
#cheek_crop=image.crop(cheek_coordinates)

In [ ]:
forehead_crop

In [ ]:
forehead_data=np.array(forehead_crop)

In [ ]:
forehead_data

array([[[206, 160, 126],
        [209, 163, 129],
        [211, 165, 131],
        ...,
        [233, 192, 160],
        [233, 192, 160],
        [232, 191, 159]],

       [[207, 161, 127],
        [210, 164, 130],
        [211, 165, 131],
        ...,
        [233, 193, 158],
        [233, 193, 158],
        [234, 191, 157]],

       [[208, 162, 128],
        [211, 165, 131],
        [212, 166, 132],
        ...,
        [233, 193, 158],
        [235, 192, 158],
        [234, 191, 157]],

       ...,

       [[207, 161, 127],
        [211, 165, 131],
        [211, 165, 131],
        ...,
        [228, 189, 158],
        [228, 187, 157],
        [233, 192, 162]],

       [[205, 159, 125],
        [209, 163, 129],
        [210, 164, 130],
        ...,
        [219, 182, 153],
        [219, 180, 151],
        [224, 185, 156]],

       [[202, 156, 122],
        [207, 161, 127],
        [209, 163, 129],
        ...,
        [193, 157, 131],
        [175, 138, 112],
        [164, 127, 101]]

In [ ]:
def colour_average1(image):
  data=np.array(image)
  shape=data.shape
  rows=shape[0]
  final_list=[]
  c=0
  for iterator in range(rows):
    for x in data[iterator]:
      list1=[x[0], x[1], x[2]]
      final_list.append(list1)
  df=pd.DataFrame(final_list, columns=['R', 'G', 'B'])
  X=copy.copy(df)
  X+=256
  for i in range(len(X["R"])):
    if X["R"][i]>256:
      c=c+1
      X["R"][i]=df["R"][i]
  for i in range(len(X["G"])):
    if X["G"][i]>256:
      X["G"][i]=df["G"][i]
  for i in range(len(X["B"])):
    if X["B"][i]>256:
      X["B"][i]=df["B"][i]
  Red=X["R"]
  Green=X["G"]
  Blue=X["B"]
  rm=np.mean(Red)
  gm=np.mean(Green)
  bm=np.mean(Blue)
  rs=np.std(Red)
  gs=np.std(Green)
  bs=np.std(Blue)
  red_trim=stats.tmean(Red, (rm-rs, rm+rs))
  green_trim=stats.tmean(Green, (gm-gs, gm+gs))
  blue_trim=stats.tmean(Blue, (bm-bs, bm+bs))
  area=len(Red)
  return [red_trim, green_trim, blue_trim, area]

In [ ]:
def colour_average(image):
  data=np.array(image)
  shape=data.shape
  rows=shape[0]
  red=[]
  green=[]
  blue=[]
  r=[]
  g=[]
  b=[]
  c=0
  for iterator in range(rows):
    for x in data[iterator]:
      red.append(x[0])
      green.append(x[1])
      blue.append(x[2])
  rm=np.mean(red)
  gm=np.mean(green)
  bm=np.mean(blue)
  rs=np.std(red)
  gs=np.std(green)
  bs=np.std(blue)
  for item in red:
    if item>=rm and item<=rm+rs:
      r.append(item)
  r=[int(x)*int(x) for x in r]
  #root mean square
  for item in green:
    if item>=gm and item<=gm+gs:
      g.append(item)
  g=[int(x)*int(x) for x in g]
  for item in blue:
    if item>=bm and item<=bm+bs:
      b.append(item)
  b=[int(x)*int(x) for x in b]
  red_trim=math.sqrt(sum(r)/len(r))
  green_trim=math.sqrt(sum(g)/len(g))
  blue_trim=math.sqrt(sum(b)/len(b))
  area=min(len(r), len(g), len(b))
  return [red_trim, green_trim, blue_trim, area]

In [ ]:
f=colour_average (forehead_crop)
g=colour_average (nose_crop)
#h=colour_average (cheek_crop)
m=[]

In [ ]:
f

[237.4628215950265, 205.6523963565912, 175.93102793924754, 267]

In [ ]:
def weighted_average (list1, list2, list3):
  list_ans=[]
  area1=list1[3]
  area2=list2[3]
  #area3=list3[3]
  for i in range(3):
    #ans=(list1[i]*area1 + list2[i]*area2 + list3[i]*area3)/(area1+area2+area3)
    ans=(list1[i]*area1 + list2[i]*area2)/(area1+area2)
    list_ans.append(ans)
  return list_ans


In [ ]:
weighted_average(f,g,m)

[237.57878765907392, 203.68228987147305, 174.14486521324295]

In [ ]:
#fdata=np.array(forehead_crop)

In [ ]:
#fdata.shape

(111, 238, 3)

In [ ]:
"""def colour_averagehsv(image):
  data=np.array(image)
  shape=data.shape
  rows=shape[0]
  hue=[]
  saturation=[]
  value=[]
  hsv=[]
  r=[]
  g=[]
  b=[]
  c=0
  for iterator in range(rows):
    for x in data[iterator]:
      (red,green,blue)=(x[0]/255, x[1]/255, x[2]/255)
      (h,s,v)=colorsys.rgb_to_hsv(red,green,blue)
      (h, s, v) = (int(h * 179), int(s * 255), int(v * 255))
      hue.append(h)
      saturation.append(s)
      value.append(v)
  rm=np.mean(hue)
  gm=np.mean(saturation)
  bm=np.mean(value)
  rs=np.std(hue)
  gs=np.std(saturation)
  bs=np.std(value)
  for item in hue:
    if item>=rm and item<=rm+rs:
      r.append(item)
      c=c+1
  r=[float(x)*float(x) for x in r]
  for item in saturation:
    if item>=gm and item<=gm+gs:
      g.append(item)
  g=[float(x)*float(x) for x in g]
  for item in value:
    if item>=bm and item<=bm+bs:
      b.append(item)
  b=[float(x)*float(x) for x in b]
  red_trim=math.sqrt(sum(r)/len(r))
  green_trim=math.sqrt(sum(g)/len(g))
  blue_trim=math.sqrt(sum(b)/len(b))
  area=min(len(r), len(g), len(b))
  return [int(red_trim), int(green_trim), int(blue_trim), area]"""

In [ ]:
#colour_averagehsv(forehead_crop)

[18, 85, 185, 11]